In [5]:
#https://github.com/chrisjmccormick/LSA_Classification

import pickle
import time
import numpy as np
import pandas as pd
import os
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from nltk.corpus import stopwords
from pylab import *
from bs4 import BeautifulSoup
import random
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords 
import string
from nltk.tag import pos_tag


TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

token_dict = {}
stemmer = PorterStemmer()
###############################################################################
#  Load the raw text dataset.
###############################################################################

print("Loading dataset...")

# The raw text dataset is stored as tuple in the form:
# (X_train_raw, y_train_raw, X_test_raw, y_test)
# The 'filtered' dataset excludes any articles that we failed to retrieve
# fingerprints for.
file_path='D:/Kalpana_II/reuters21578/'

stop_words = set(stopwords.words('english'))
stop_words.update(['around','need','addess','due','goes','go','previous','next','kind','like','week','year','month','led','previously','daily','day',
                   '<br>','</br>','<br/>','peter','via','based','base','upon','currently','pm','am','several','also','monday','tuesday',
                   'wednesday','thursday','friday','sunday', 'recent', 'since', 'per', 'look', 'see', 'set', 'still', 
                   'following', 'max'  ,'&nbsp;','&nbsp','nbsp','soon','please','need', ',', '"', "'",  '!', ':', ';',
                   '(', ')', '[', ']', '{', '}', '?',',','/','"','$',"'d","'s","'ve","+at","+-ep", "<",">", "br",".",
                    '*', '***','***', '&', '+','{', '}', '~', '#',"''","=+","***doe"]) # remove it if you need punctuation 
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize_org(text):
    soup = BeautifulSoup(text)
    text_1=soup.get_text()
    text_1=text_1.replace('\d+', '')
    tokens = nltk.word_tokenize(text_1)
    html_tag = [i for i in tokens if i  in remove_tags(i) and (len(i)>=3)] 
    digit_rem=[i for i in html_tag if not i.isdigit()]    
    stems = stem_tokens(digit_rem, stemmer)
    return stems

def tokenize(text):
    text= str(text).lower()    
    text = text.replace("'", "")     
#     text = replace_all(text,replace_dict)        
    text = re.sub('(\d{1,4})[/.-:](\d{1,2})[/.-:](\d{0,4})', ' ', text).strip()      
    text = re.sub(r"\b[-+]?[\d.]+\b", ' ', text)    
    text = re.sub('[^A-Za-z0-9]+', ' ', text)        
#     text = replace_all(text,replace_dict) 
    words = [word for word in nltk.word_tokenize(text)] #for sent in nltk.sent_tokenize(text)      
    words = [w for w in words if w not in stop_words and len(w)>=2 ]          
    words = [stemmer.stem(i) for i in words]   
    return words

def build_LDA_Model(tfs):
    lda_model = LatentDirichletAllocation(n_topics=5,               # Number of topics
                                          max_iter=10,               
    # Max learning iterations
                                          learning_method='online',   
                                          random_state=100,          
    # Random state
                                          batch_size=128,            
    # n docs in each learning iter
                                          evaluate_every = -1,       
    # compute perplexity every n iters, default: Don't
                                          n_jobs = -1,               
    # Use all available CPUs
                                         )
    lda_output = lda_model.fit_transform(tfs)
    print(lda_model)  # Model attributes
    
    #Analyzed model performance with perplexity & log-likelihood
    # Log Likelyhood: Higher the better
    print("Log Likelihood: ", lda_model.score(tfs))
    # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
    print("Perplexity: ", lda_model.perplexity(tfs))
    # See model parameters
    print(lda_model.get_params())

def Gridserachcv_LDA(tfs):
    # Define Search Param
    search_params = {'n_topics': [3,5,6], 'learning_decay': [.5, .7, .9]}
    # Init the Model
    lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
    # Init Grid Search Class
    model = GridSearchCV(lda, param_grid=search_params)
    # Do the Grid Search
    model.fit(tfs)
    GridSearchCV(cv=None, error_score='raise',
           estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                 evaluate_every=-1, learning_decay=0.7, learning_method=None,
                 learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
                 mean_change_tol=0.001, n_topics=10, n_jobs=1,
#                  n_topics=None, perp_tol=0.1, random_state=None,
                 topic_word_prior=None, total_samples=1000000.0, verbose=0),
           fit_params=None, iid=True, n_jobs=1,
           param_grid={'n_topics': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
           pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
           scoring=None, verbose=0)
    
    # Best Model
    best_lda_model = model.best_estimator_
    # Model Parameters
    print("Best Model's Params: ", model.best_params_)
    # Log Likelihood Score
    print("Best Log Likelihood Score: ", model.best_score_)
    # Perplexity
    print("Model Perplexity: ", best_lda_model.perplexity(tfs))
    return best_lda_model

def classify_doc_topics(docnames,tfs, feature_names,best_lda_model):
    # Create Document — Topic Matrix
    lda_output = best_lda_model.transform(tfs)
    # column names
    topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_topics)]
    # index names
#     docnames = ["Doc" + str(i) for i in range(len(data))]
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic
    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)
    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)
    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    df_document_topics
    
    # Topic-Keyword Matrix
    df_topic_keywords = pd.DataFrame(best_lda_model.components_)
    # Assign Column and Index
    df_topic_keywords.columns = feature_names
    df_topic_keywords.index = topicnames
    # View
    print(df_topic_keywords.head())
    
    return df_document_topics,topicnames,lda_output

# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords



file_name=[]
def read_dir_into_df(fpath):
    
    dirListing = os.listdir(fpath)
    bloblist = []
    
    simi_result=[]
    df=pd.DataFrame()
    corpus = []
    for item in dirListing:
        if ".txt" in item:
            text_file = (fpath+'\\'+item)
            file_name.append(item)
            with open (text_file) as f:
                corpus.append(f.read()) 
                f.close()
    df_text = pd.DataFrame(corpus,columns=['text']) 
    df_text['doc_name'] = list(file_name)
    df_text.set_index('doc_name', inplace=True)
    print(file_name)
    return df_text,file_name

df_result,file_name = read_dir_into_df(file_path)
print(df_result)

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=stop_words,ngram_range=(1, 1))
#feature_names = tfidf.get_feature_names()
tfs = tfidf.fit_transform((df_result['text']))#, index=df_result['text'])#, columns=feature_names)
# print(tfs)
scores = np.asarray(tfs.sum(axis=0)).ravel()
scores_lst = [i for i,v in enumerate(scores) if v>0]
feature_names = tfidf.get_feature_names()
dfFinal = pd.DataFrame(tfs.toarray())

a1=(pd.DataFrame(tfs.A, columns=(tfidf.get_feature_names()))) #print matrix
a1.index = df_result.index
print(a1)


#calling LDA model
build_LDA_Model(tfs)
best_lda_model = Gridserachcv_LDA(tfs)
doc_topic,topicnames,lda_output = classify_doc_topics(file_name,tfs, feature_names, best_lda_model)
topic_keywords = show_topics(tfidf, best_lda_model, 20)

# topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

print('----------------------')
print(type(lda_output))#nd_array
print(lda_output.shape[0])#7
print(lda_output.shape[1])#
print('----------------------')
# Splitting the dataset into the Training set and Test set
# labels=[]
# from sklearn.naive_bayes import MultinomialNB
# sv=MultinomialNB()
# # labels=[0]
# for i in range(0, 3):
#     labels.append(0)
# sv.fit(lda_output,labels)
# predictclass = sv.predict(lda_output)

# testLables=[0]
# from sklearn import metrics
# yacc=metrics.accuracy_score(testLables,predictclass)
# print (yacc)
# n=dataset.target[:20]
# print (n)

Loading dataset...
['all-exchanges-strings.lc.txt', 'all-orgs-strings.lc.txt', 'all-people-strings.lc.txt', 'all-places-strings.lc.txt', 'all-topics-strings.lc.txt', 'cat-descriptions_120396.txt', 'feldman-cia-worldfactbook-data.txt']
                                                                                 text
doc_name                                                                             
all-exchanges-strings.lc.txt        amex\nase\nasx\nbiffex\nbse\ncboe\ncbt\ncme\nc...
all-orgs-strings.lc.txt             adb-africa\nadb-asia\naibd\naid\nanrpc\nasean\...
all-people-strings.lc.txt           abdel-hadi-kandeel\nalfonsin\nalhaji-abdul-ahm...
all-places-strings.lc.txt           afghanistan\nalbania \nalgeria \namerican-samo...
all-topics-strings.lc.txt           acq\nalum\naustdlr\naustral\nbarley\nbfr\nbop\...
cat-descriptions_120396.txt         \nSome notes on the Reuters Categories\nDavid ...
feldman-cia-worldfactbook-data.txt  This is a set of Prolog assertions for fa

ValueError: Found input variables with inconsistent numbers of samples: [7, 3]

In [ ]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Import the dataset
dataset = pd.read_csv('LDA_Data.csv')
X = dataset.iloc[:, 0:13].values
y = dataset.iloc[:, 13].values

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Feature Scaling to Dataset
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Implement LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 2)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

# Train Logistic Regression with LDA
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict Results of Regression with LDA
y_pred = classifier.predict(X_test)